In [2]:
import numpy as np
import sympy as sp
from sympy import ntheory
from sympy.ntheory import continued_fraction_periodic
import math

In [3]:
def classRepresentatives(trace): # gives quadratic forms of given trace
    bound = math.floor(trace**2-4)
    quadraticForms = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            quadraticForms+=representatives(D)
        elif D%4 == 1:
            quadraticForms+=representatives(D)
        i = i + 1
    return [Q for Q in quadraticForms if tr(Q)==np.round(trace)]

def representatives(D):
    reps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                reps.append(Q)
            else:
                continue
    return reps

def classCFReps(trace): # gives continued fractions of a given trace
    bound = math.floor(trace**2-4)
    cfList = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            cfList+=cfReps(D)
        elif D%4 == 1:
            cfList+=cfReps(D)
        i = i + 1
    return [cf for cf in cfList if cfTrace(cf)==np.round(trace)]

def cfReps(D):
    fracReps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                fracReps.append(frac)
            else:
                continue
    return fracReps

def permutationsOf(a):
    perms = []
    if len(a)%2==0:
        for n in range(0,len(a),2):
            perms.append(a[n:]+a[:n])
    else:
        for n in range(0,len(a)):
            perms.append(a[n:]+a[:n])
    return perms

def disc(Q):
    return Q[1]**2-4*Q[0]*Q[2]

def areEquivalent(Q,W):
    return ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] in permutationsOf(ntheory.continued_fraction_periodic(-W[1], 2*W[0], disc(W))[0])

def equalsQuad(L,M):
    if len(L)!=len(M):
        return False
    for l in L:
        counter = 0
        for m in M:
            if areEquivalent(m,l):
                M.remove(m)
                counter+=1
        if counter !=1:
            return False
    return True

def classNum(D):
    return len(representatives(D))

def tr(Q):
    return np.trace(quadToMat(Q))

def cfTrace(cf):
    return np.trace(cfToMat(cf))

def isASquare(n):
    for i in range(2,n):
        if n==i**2:
            return True
    return False

def gcd(a,b,c):
    return np.gcd(a,np.gcd(b,c))

def matgcd(mat):
    return gcd(mat[1,0],mat[1,1]-mat[0,0],mat[0,1])

def quadToMat(Q):
    mat = np.identity(2)
    length = 0
    for x in ntheory.continued_fraction_periodic(-Q[1], 2*Q[0],disc(Q))[0]:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)

In [4]:
def contFracToLR(contFrac):
    if len(contFrac)%2==1:
        contFrac = contFrac + contFrac
    output = []
    for i in range(len(contFrac)):
        if i%2==1: #if original cont'd frac is even length, making last one R/alternating is same as this (hopefully)
            for j in range(contFrac[i]):
                output.append(1)
        else:
            for j in range(contFrac[i]):
                output.append(0)
    return output

def recover2(ls): #sends a whole list to a cont'd fraction (not clear whether this is needed)
    output = []
    i=0
    while i<len(ls):
        counter=1
        while(i<len(ls)-1 and ls[i] == ls[i+1]):
            i+=1
            counter+=1
        i+=1
        output.append(counter)
    if (len(output)%2!=0):
        output[len(output)-1]+=output[0]
        output = output[1:]
    return output

def indices(lrSeq): #positions of R's (R=1)
    return [i for i in range(len(lrSeq)) if lrSeq[i]==1]

def pairs(lrSeq): #pairs of R's on which to split
    output = []
    for i in indices(lrSeq):
        for j in indices(lrSeq):
            if i==j:
                continue
            else:
                output.append((i,j))
    return output

def split(contFrac): #splits a cont'd fraction on every possible pair of R's into 2 groups of R's and L's (in 0's and 1's)
    output = []
    converted = contFracToLR(contFrac)
    n = len(converted)
    splitPoints = pairs(converted)
    for i,j in splitPoints:
        temp = converted[i:]+converted[:i]
        temp1 = temp[1:(j-i)%n]
        temp2 = inv(temp[(j-i)%n+1:]) #inv(temp[(j-i)%n+1:])
        output.append((tuple(temp1),tuple(temp2)))
    return output

def printCon(lrSeq): #R = 1, L = 0
    a = ''
    for i in lrSeq:
        if i==1:
            a+= 'R'
        else:
            a+= 'L'
    return a

def printConLs(ls):
    printList = []
    for L in ls:
        printList.append(printCon(L))
    return printList

#def getDict(contFrac):
#    output = []
#    fracs = []
#    for t in range(1+2*e):
#        tr = trace-e+t
#        fracs += [ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] for Q in classRepresentatives(tr)]
#    for f in fracs:
#        output += split(f)
#    return output

#def cfListToDict(cfList):
#    outputDict = {}
#    for cf in cfList:
#        for obj in split(cf):
#            if obj[0] in outputDict:
#                outputDict[obj[0]].add(obj[1])
#            else:
#                outputDict[obj[0]] = {obj[1]}
#    return outputDict

def cfListToDicts(cfList):
    forwardDict = {}
    invDict = {}
    for cf in cfList:
        for obj in split(cf):
            if obj[0] in forwardDict:
                forwardDict[obj[0]].add(obj[1])
            else:
                forwardDict[obj[0]] = {obj[1]}
            if obj[1] in invDict:
                invDict[obj[1]].add(obj[0])
            else:
                invDict[obj[1]] = {obj[0]}
    return forwardDict,invDict

def inv(lrSeq): #inverts a list of 0's and 1's (R's and L's)
    return tuple((np.array(lrSeq[::-1])-1)**2)

def pairToFrac(s1,s2):
    out = list(s1)+[1]+list(inv(s2))+[1]
    return tuple(recover2(out))

#def getPants(cfList): #this clearly doesn't work
#    allPairs = cfListToDict(cfList)
#    output,fracs = [],[]
#    for s0,s1 in allPairs:
#        temp1,temp2 = set([tuple(a[1]) for a in allPairs if a[0]==s0]), set([tuple(a[1]) for a in allPairs if a[0]==s1])
#        s  = temp1.intersection(temp2)
#        for s2 in s:
#            s2 = list(s2)
#            output.append([s0,s1,s2])
#            fracs = [pairToFrac(s0,inv(s1)),pairToFrac(s1,inv(s2)),pairToFrac(s2,inv(s0))]
#    return fracs

def getPants(cfList):
    dictPair = cfListToDicts(cfList)
    forwardDict = dictPair[0]
    invDict = dictPair[1]
    pants = set()
    for s0 in forwardDict:
        for s1 in forwardDict[s0]:
            if s1 in forwardDict and s0 in invDict:
                for s2 in forwardDict[s1].intersection(invDict[s0]):
                    pants.add((pairToFrac(s0,s1),pairToFrac(s1,s2),pairToFrac(s2,s0)))
    return pants

def pantsInTraceRange(traceMin,traceMax):
    cfList = []
    for trace in range(traceMin,traceMax+1):
        cfList += classCFReps(trace)
    return getPants(cfList)

def cfToMat(cf):
    mat = np.identity(2)
    length = 0
    for x in cf:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)

In [22]:
getPants([[1,1,2,2,2,1],[1,1,3,3],[1,8]])

{((1, 1, 2, 2, 2, 1), (1, 1, 3, 3), (1, 8)),
 ((1, 1, 3, 3), (1, 8), (1, 1, 2, 2, 2, 1)),
 ((1, 8), (1, 1, 2, 2, 2, 1), (1, 1, 3, 3)),
 ((1, 8), (3, 3, 1, 1), (2, 1, 1, 1, 2, 2)),
 ((2, 1, 1, 1, 2, 2), (1, 8), (3, 3, 1, 1)),
 ((3, 3, 1, 1), (2, 1, 1, 1, 2, 2), (1, 8))}

In [23]:
pantsInTraceRange(50,50)

{((1, 1, 1, 1, 1, 5), (2, 1, 1, 9), (6, 2, 2, 1)),
 ((1, 1, 1, 1, 1, 5), (2, 2, 6, 1), (1, 1, 2, 9)),
 ((1, 1, 1, 5, 1, 1), (2, 1, 6, 2), (2, 1, 1, 9)),
 ((1, 1, 2, 9), (1, 1, 1, 1, 1, 5), (2, 2, 6, 1)),
 ((1, 1, 2, 9), (6, 1, 2, 2), (1, 5, 1, 1, 1, 1)),
 ((1, 2, 3, 4), (1, 2, 3, 4), (1, 2, 3, 4)),
 ((1, 3, 1, 1, 2, 1), (12, 4), (3, 16)),
 ((1, 4, 3, 2), (1, 4, 3, 2), (1, 4, 3, 2)),
 ((1, 5, 1, 1, 1, 1), (1, 1, 2, 9), (6, 1, 2, 2)),
 ((2, 1, 1, 3, 1, 1), (3, 16), (12, 4)),
 ((2, 1, 1, 9), (1, 1, 1, 5, 1, 1), (2, 1, 6, 2)),
 ((2, 1, 1, 9), (6, 2, 2, 1), (1, 1, 1, 1, 1, 5)),
 ((2, 1, 6, 2), (2, 1, 1, 9), (1, 1, 1, 5, 1, 1)),
 ((2, 2, 6, 1), (1, 1, 2, 9), (1, 1, 1, 1, 1, 5)),
 ((3, 2, 1, 4), (3, 2, 1, 4), (3, 2, 1, 4)),
 ((3, 4, 1, 2), (3, 4, 1, 2), (3, 4, 1, 2)),
 ((3, 16), (1, 3, 1, 1, 2, 1), (12, 4)),
 ((3, 16), (12, 4), (2, 1, 1, 3, 1, 1)),
 ((6, 1, 2, 2), (1, 5, 1, 1, 1, 1), (1, 1, 2, 9)),
 ((6, 2, 2, 1), (1, 1, 1, 1, 1, 5), (2, 1, 1, 9)),
 ((6, 8), (6, 8), (6, 8)),
 ((12, 4), (2, 1,

In [24]:
conv = contFracToLR([1,2,3,4])
print(conv)
temp = conv[6:] + conv[:6]
print(temp)
temp1 = temp[1:(1-6)%len(conv)]
print(temp1)
temp2 = temp[(1-6)%len(conv)+1:]
print(temp2)

[0, 1, 1, 0, 0, 0, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 0, 0, 0]
[1, 1, 1, 0]
[1, 0, 0, 0]


In [25]:
print(contFracToLR([1,2,3,4]))
print(printCon(contFracToLR([1,2,3,4])))
print('\n')
for obj in split([1,2,3,4]):
    print(printCon(obj[0]),printCon(obj[1]))

[0, 1, 1, 0, 0, 0, 1, 1, 1, 1]
LRRLLLRRRR


 RLLLLRRR
RLLL RLLL
RLLLR RLL
RLLLRR RL
RLLLRRR R
LLLRRRRL 
LLL LRLLL
LLLR LRLL
LLLRR LRL
LLLRRR LR
RRRL RRRL
RRRLR RRR
 RRRLLRLL
R RRRLLRL
RR RRRLLR
RRL LRRRL
RRLR LRRR
RRLRRLLL 
 LRRRLLRL
R LRRRLLR
RL LLRRRL
RLR LLRRR
RLRRLLL L
RLRRLLLR 
 LLRRRLLR
L LLLRRRL
LR LLLRRR
LRRLLL LL
LRRLLLR L
LRRLLLRR 


In [26]:
getPants([[1,1,2,2,2,1],[1,1,3,3],[1,8],[1,2,2,2,1,1],[3,3,1,1],[8,1]])

{((1, 1, 2, 2, 2, 1), (1, 1, 3, 3), (1, 8)),
 ((1, 1, 3, 3), (1, 8), (1, 1, 2, 2, 2, 1)),
 ((1, 8), (1, 1, 2, 2, 2, 1), (1, 1, 3, 3)),
 ((1, 8), (3, 3, 1, 1), (2, 1, 1, 1, 2, 2)),
 ((2, 1, 1, 1, 2, 2), (1, 8), (3, 3, 1, 1)),
 ((3, 3, 1, 1), (2, 1, 1, 1, 2, 2), (1, 8))}

In [27]:
cfListToDicts([[1,1,2,2,2,1],[1,1,3,3],[1,8],[1,2,2,2,1,1],[3,3,1,1],[8,1]])

({(0, 0): {(0, 1, 0, 1, 0), (0, 1, 1, 0, 1), (1, 0, 1, 1, 0)},
  (0, 0, 1): {(0, 1, 0, 1), (1, 0, 1, 1)},
  (0, 0, 1, 1, 0, 0): {(1,)},
  (1, 0, 0, 1, 0): {(1, 1)},
  (): {(0, 0, 0, 0, 0, 0, 1),
   (0, 0, 0, 0, 0, 1, 0),
   (0, 0, 0, 0, 1, 0, 0),
   (0, 0, 0, 1, 0, 0, 0),
   (0, 0, 1, 0, 0, 0, 0),
   (0, 1, 0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0, 1),
   (1, 0, 0, 0, 0, 0, 0),
   (1, 0, 1, 0, 0, 1, 1),
   (1, 1, 0, 0, 1, 0, 1),
   (1, 1, 0, 1, 0, 1, 1),
   (1, 1, 1, 0, 1, 0)},
  (1, 0, 0): {(1, 0, 1, 0), (1, 1, 0, 1)},
  (0, 0, 1, 0): {(0, 1, 1)},
  (0, 0, 1, 0, 1, 0, 0): {()},
  (0,): {(0, 0, 0, 0, 0, 0),
   (0, 0, 1, 1, 1),
   (0, 1, 1, 0, 0, 1),
   (1, 0, 0, 1, 1, 0),
   (1, 1, 0, 0, 1, 1)},
  (0, 1, 0, 0): {(1, 1, 0)},
  (0, 1, 0, 0, 1): {(1, 1)},
  (0, 0, 0): {(1, 0, 0)},
  (0, 0, 0, 1): {(1, 0)},
  (0, 0, 0, 1, 1): {(1,)},
  (1, 1, 0): {(0, 0, 0, 0), (1, 1, 1)},
  (1,): {(0, 0, 0, 0, 0, 1),
   (0, 0, 0, 0, 1, 0),
   (0, 0, 0, 1, 0, 0),
   (0, 0, 1, 0, 0, 0),
   (0, 1, 0, 0, 0, 0),
   (

In [28]:
cfListToDicts([[1,1,2,2,2,1]])

({(0, 0): {(0, 1, 1, 0, 1), (1, 0, 1, 1, 0)},
  (0, 0, 1): {(1, 0, 1, 1)},
  (0, 0, 1, 1, 0, 0): {(1,)},
  (1, 0, 0, 1, 0): {(1, 1)},
  (): {(1, 1, 0, 1, 0, 1, 1)},
  (1, 0, 0): {(1, 1, 0, 1)},
  (0, 0, 1, 0): {(0, 1, 1)},
  (0, 0, 1, 0, 1, 0, 0): {()},
  (0,): {(1, 1, 0, 0, 1, 1)},
  (0, 1, 0, 0): {(1, 1, 0)},
  (0, 1, 0, 0, 1): {(1, 1)}},
 {(1, 0, 1, 1, 0): {(0, 0)},
  (1, 0, 1, 1): {(0, 0, 1)},
  (1,): {(0, 0, 1, 1, 0, 0)},
  (1, 1): {(0, 1, 0, 0, 1), (1, 0, 0, 1, 0)},
  (1, 1, 0, 1, 0, 1, 1): {()},
  (1, 1, 0, 1): {(1, 0, 0)},
  (0, 1, 1): {(0, 0, 1, 0)},
  (): {(0, 0, 1, 0, 1, 0, 0)},
  (0, 1, 1, 0, 1): {(0, 0)},
  (1, 1, 0, 0, 1, 1): {(0,)},
  (1, 1, 0): {(0, 1, 0, 0)}})

In [29]:
cfListToDicts([[2,1,1,1,2,2]]) == cfListToDicts([[2,2,2,1,1,1]])

True